In [1]:
import torch
import torch.nn as nn   
import torch.optim as optim
import numpy as np

'apple!' : input_data => apple apple, output_data ==> pple! 

In [5]:
input_str='apple'
label_str='pple!' # 맨 마지막 글자인 '!'는 예측할 필요가 없으므로 잘라냄
char_set=sorted(list(set(input_str+label_str))) # input과 label을 합쳐서 중복을 제거한 문자 집합 생성
print(char_set) # 문자 코드 순서로 출력 
char_set_size=len(char_set)
print(char_set_size)

['!', 'a', 'e', 'l', 'p']
5


In [6]:
input_size=5 # one-hot size
hidden_size=5 # output from the LSTM. 5 to directly predict one-hot
output_size=5  # final output size (RNN or softmax의 결과로 나오는 output size) 
learning_rate=0.1 # learning rate means how much the model learns in each step 

In [7]:
char_to_index=dict((c,i) for i, c in enumerate(char_set)) # 문자 집합에 대해 index를 부여 (key: 문자, value: index)

""" char_list=[c for c in char_set] # list comprehension을 이용하여 문자 집합에 대해 index를 부여
char_list=[]
for c in char_set:
    char_list.append(c) """

print(char_to_index) 

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}


In [10]:
char_to_index.items()
char_to_index.values()
char_to_index.keys()

dict_keys(['!', 'a', 'e', 'l', 'p'])

In [11]:
# 참고 - 이런식으로도 가능

index_to_char={}
for key, value in char_to_index.items(): # key와 value를 바꿔 index to char를 생성
    index_to_char[value]=key

print(index_to_char)

{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [12]:
x_data=[char_to_index[c] for c in input_str] # x_data는 입력에 해당되는 문자들에 대한 index 값을 x -> apple
y_data=[char_to_index[c] for c in label_str] # y_data는 출력에 해당되는 문자들에 대한 index -> pple! 

print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [14]:
x_one_hot=[np.eye(char_set_size)[x] for x in x_data] # char_set size만큼의 단위행렬을 만들고 x_data에 해당하는 index에만 1을 부여하여 one-hot 벡터를 생성
#np.eye()는 단위행렬을 만들어주는 함수 
# one_hot encoding은 단어 집합의 크기를 벡터의 차원으로 하고, 표현하고 싶은 단어의 인덱스에 1의 값을 부여하고, 다른 인덱스에는 0을 부여하는 단어의 벡터 표현 방식
print(x_one_hot)

# 결과값 -> apple


[array([0., 1., 0., 0., 0.]), array([0., 0., 0., 0., 1.]), array([0., 0., 0., 0., 1.]), array([0., 0., 0., 1., 0.]), array([0., 0., 1., 0., 0.])]


In [15]:
X=torch.FloatTensor(x_one_hot) # one-hot vector를 float tensor로 변환
Y=torch.LongTensor(y_data) # CrossEntropyLoss를 사용하기 위해 Y를 long tensor로 변환

print(X)
print(Y)

tensor([[0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 1.],
        [0., 0., 0., 1., 0.],
        [0., 0., 1., 0., 0.]])
tensor([4, 4, 3, 2, 0])


C:\Users\it\AppData\Local\Temp\ipykernel_10440\2197340700.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X=torch.FloatTensor(x_one_hot) # one-hot vector를 float tensor로 변환


In [17]:
#nn.RNN(input_size, hidden_size, output_size)
class Net (nn.Module):
    def __init__(self, input_size, hidden_size, output_size): 
        super(Net, self).__init__() # nn.Module의 속성들을 가지고 초기화
        self.rnn=nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True) # RNN 모듈을 사용
        # batch_first= input 데이터의 첫번째 차원이 batch size임을 알려줌. batch size가 첫번째 차원이 아니라면 False로 설정
        # batch size, sequence length, 특징 차원 순서로 데이터를 입력받음
        # input 형태가 (32, 10, 50) 이라면 batch_first=True로 설정하면 (32, 10, 50)으로 입력받음 
        self.fc=nn.Linear(hidden_size, output_size, bias=True)  # RNN의 output을 input으로 받아 output_size를 출력하는 linear layer를 생성

    def forward(self, x): # forward 함수를 정의 -> forward는 모델을 학습할 때 데이터를 입력받아 forward 연산을 진행시키는 함수
        x, _status=self.rnn(x) # RNN의 output과 hidden state를 반환
        # RNN은 두 개의 output을 반환
        x=self.fc(x)
        return x


In [18]:
model=Net(input_size, hidden_size, output_size) # model을 생성
criterion=nn.CrossEntropyLoss() # CrossEntropyLoss를 사용
optimizer=optim.Adam(model.parameters(), learning_rate) # Adam optimizer를 사용하는 이유는 gradient vanishing 문제를 해결하기 위함


In [21]:
output = model(X) # model에 X를 입력하여 output을 얻음 (X: apple)
print(output) # output을 출력 -> 5개의 문자에 대한 예측값이 출력됨 
print(output.size()) # output size를 출력 -> 4,4,3,2,0 (pple!)으로 출력되게 학습시켜야함 

tensor([[-0.2656,  0.3877,  0.1062,  0.0855, -0.0290],
        [-0.4354,  0.3205, -0.0681,  0.1284,  0.1280],
        [-0.3597,  0.2890, -0.0095,  0.0618,  0.0763],
        [ 0.0614,  0.0456,  0.0463, -0.0600, -0.0548],
        [ 0.0391,  0.0652, -0.0469,  0.0720,  0.0185]],
       grad_fn=<AddmmBackward0>)
torch.Size([5, 5])
